### Introduction 

Notebook explores how a LLM can call tools <br>
https://python.langchain.com/docs/how_to/tool_results_pass_to_model/
https://github.com/langchain-ai/langchain/blob/master/docs/docs/tutorials/agents.ipynb


### Setting Up Model

Using llama3.1 for quick development purposed.
Need to do ```ollama pull MODELNAME``` before running.

In [42]:
from langchain.chat_models import init_chat_model
# from langchain_tavily import TavilySearch
# from langgraph.checkpoint.memory import MemorySaver
# from langgraph.prebuilt import create_react_agent

llm = init_chat_model(
    model="ollama:llama3.1",        # or "gpt-4o-mini", "ollama/llama3.1", etc.
    temperature=0.2  # lower = more deterministic
)


### Defining Tools

In [ ]:
# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.

from langchain_core.tools import tool
#The decorator wraps in langchains type so it becomes runnable wiht .invoke(..)
@tool
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b


tools = [add, multiply]




### Testing: Can LLM see tools?

LLM can not call tools in lang chain, only request.

In [44]:
llm_with_tools = llm.bind_tools(tools) #llm_with_tools is a new wrapped llm

query = "What is 3 * 12?"

llm_with_tools.invoke(query) #



AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-08-24T22:41:48.407425Z', 'done': True, 'done_reason': 'stop', 'total_duration': 13486425584, 'load_duration': 12192829334, 'prompt_eval_count': 254, 'prompt_eval_duration': 773944250, 'eval_count': 22, 'eval_duration': 518181209, 'model_name': 'llama3.1'}, id='run--db5e283e-d191-42ad-aa56-f52e4583c0b4-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'a56c326f-93cd-46be-a79a-207f27db586e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 254, 'output_tokens': 22, 'total_tokens': 276})

### Full Agentic Tool Use by LLM

Set up quick chat history and invoke model

In [45]:
from langchain_core.messages import HumanMessage
#message classes on lang chain inlcude human massage ,ai message, system message, and tool message
query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages) #llm_with_tools looks at history(currently only 1 human message) and then builds prompt
print("AI MESSAEGE CALLS")
print(ai_msg)
print("JUST AI TOOL CALLS")
print(ai_msg.tool_calls) 
#Now we add the bots message to the chat history
messages.append(ai_msg)

AI MESSAEGE CALLS
content='' additional_kwargs={} response_metadata={'model': 'llama3.1', 'created_at': '2025-08-24T22:41:49.622484Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1187052292, 'load_duration': 42706417, 'prompt_eval_count': 264, 'prompt_eval_duration': 102223542, 'eval_count': 43, 'eval_duration': 1041535375, 'model_name': 'llama3.1'} id='run--3f8d2bee-2aff-4a8d-a783-513efdaf68a7-0' tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'f09d3337-0574-4034-9d82-900b48ce1125', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': '3d89bbff-24d7-4032-b7b6-a4c12091d76f', 'type': 'tool_call'}] usage_metadata={'input_tokens': 264, 'output_tokens': 43, 'total_tokens': 307}
JUST AI TOOL CALLS
[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'f09d3337-0574-4034-9d82-900b48ce1125', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': '3d89bbff-24d7-4032-b7b6-a4c12091d76f', 'type': 'tool_call'}]


#### Acually call the tools and give response back

In [46]:
for tool_call in ai_msg.tool_calls: #Actually running all the tool calls ai requested in last cell
    tools_dict = {"add": add, "multiply": multiply}
    selected_tool = tools_dict[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg) #Add the tool message to chat history

messages




[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-08-24T22:41:49.622484Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1187052292, 'load_duration': 42706417, 'prompt_eval_count': 264, 'prompt_eval_duration': 102223542, 'eval_count': 43, 'eval_duration': 1041535375, 'model_name': 'llama3.1'}, id='run--3f8d2bee-2aff-4a8d-a783-513efdaf68a7-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'f09d3337-0574-4034-9d82-900b48ce1125', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': '3d89bbff-24d7-4032-b7b6-a4c12091d76f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 264, 'output_tokens': 43, 'total_tokens': 307}),
 ToolMessage(content='36', name='multiply', tool_call_id='f09d3337-0574-4034-9d82-900b48ce1125'),
 ToolMessage(content='60', name='add', tool_call_id='3d89bbff-

#### Give final response

In [47]:
ans = llm_with_tools.invoke(messages)
ans

AIMessage(content='The result of 3 * 12 is 36. The result of 11 + 49 is 60.', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-08-24T22:41:50.491291Z', 'done': True, 'done_reason': 'stop', 'total_duration': 854849542, 'load_duration': 28822000, 'prompt_eval_count': 130, 'prompt_eval_duration': 241417875, 'eval_count': 25, 'eval_duration': 582864375, 'model_name': 'llama3.1'}, id='run--4f387296-9b21-434b-8a95-66897cfd5216-0', usage_metadata={'input_tokens': 130, 'output_tokens': 25, 'total_tokens': 155})

#### Just the final output

In [48]:
(ans.content)

'The result of 3 * 12 is 36. The result of 11 + 49 is 60.'